# YOLOv8-Nano Detection Model Training
## Cattle Breed Recognition System - Stage 1: Animal Detection

This notebook trains a YOLOv8-Nano model for detecting cattle and buffaloes in images.

**Model Specifications:**
- Model: YOLOv8-Nano (smallest, fastest)
- Input Size: 416x416
- Output: Bounding box (x, y, w, h) for animal detection
- Final Size: ~5 MB (after INT8 quantization)
- Inference: ~20ms on mobile

**Author:** SIH 2025 Team
**Problem Statement:** SIH25004 - Image-based Breed Recognition for Cattle and Buffaloes of India

In [1]:
# FIX: PyTorch 2.6+ compatibility (RUN THIS FIRST!)
import torch
import functools

_original_torch_load = torch.load

@functools.wraps(_original_torch_load)
def _patched_load(f, map_location=None, pickle_module=None, *, weights_only=None, mmap=None, **kwargs):
    return _original_torch_load(f, map_location=map_location, pickle_module=pickle_module,
                                 weights_only=False, mmap=mmap, **kwargs)

torch.load = _patched_load
print("✅ PyTorch patch applied! Now run the rest of the notebook.")


✅ PyTorch patch applied! Now run the rest of the notebook.


In [2]:
# Check GPU availability
!nvidia-smi

# Install YOLOv8
!pip install ultralytics==8.0.196 -q
!pip install roboflow -q


/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 137.3 MB/s eta 0:00:00


In [3]:
import os
import yaml
import json
import shutil
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
from google.colab import drive

# Mount Google Drive for data storage
drive.mount('/content/drive')

# Set paths
BASE_PATH = '/content/drive/MyDrive/cattle_breed_recognition'
...


Mounted at /content/drive


Ellipsis

In [4]:
# Load ZIP files from Google Drive backup
import os
import shutil
import zipfile
import random

backup_folder = '/content/drive/MyDrive/cattle_breed_recognition/dataset_backup'
print("📂 Loading ZIP files from Google Drive backup...")

if os.path.exists(backup_folder):
    for f in os.listdir(backup_folder):
        if f.endswith('.zip'):
            src = os.path.join(backup_folder, f)
            dst = f'/content/{f}'
            if not os.path.exists(dst):
                size_mb = os.path.getsize(src) / (1024 * 1024)
                print(f"   Loading: {f} ({size_mb:.1f} MB)")
                shutil.copy2(src, dst)
            else:
                print(f"   Already exists: {f}")
    print("✅ ZIP files loaded!")
else:
    print("❌ Backup folder not found!")


📂 Loading ZIP files from Google Drive backup...
❌ Backup folder not found!


In [5]:
# Check Google Drive contents
import os

drive_path = '/content/drive/MyDrive'
print("📁 Google Drive contents:")
print("=" * 60)

if os.path.exists(drive_path):
    for item in sorted(os.listdir(drive_path)):
        item_path = os.path.join(drive_path, item)
        if os.path.isdir(item_path):
            print(f"   📂 {item}/")
        else:
            size_mb = os.path.getsize(item_path) / (1024 * 1024)
            print(f"   📄 {item} ({size_mb:.1f} MB)")
else:
    print("   (empty or not accessible)")

# Check if cattle_breed_recognition folder exists
cattle_path = '/content/drive/MyDrive/cattle_breed_recognition'
print("\n" + "=" * 60)
if os.path.exists(cattle_path):
    print(f"✅ cattle_breed_recognition folder exists")
    print("Contents:")
    for item in sorted(os.listdir(cattle_path)):
        print(f"   📂 {item}/")
else:
    print("❌ cattle_breed_recognition folder not found")


📁 Google Drive contents:
   📂 Classroom/
   📂 Colab Notebooks/
   📄 SHIVAM JAISWAL.png (0.3 MB)
   📄 Screenshot_20260105_210835.jpg (0.2 MB)
   📂 cattle_breed_recognition/

✅ cattle_breed_recognition folder exists
Contents:
   📂 01_yolov8_detection_training.ipynb/
   📂 02_efficientnet_classification_training.ipynb/
   📂 03_classification_training.ipynb/
   📂 data/
   📂 models/


In [6]:
# SAVE TO GOOGLE DRIVE IMMEDIATELY
import os
import shutil

# Create backup folder
backup_folder = '/content/drive/MyDrive/cattle_breed_recognition/dataset_backup'
os.makedirs(backup_folder, exist_ok=True)

print("💾 Saving ZIP files to Google Drive...")
print("=" * 60)

# Find ZIP files in /content/
zip_files = [f for f in os.listdir('/content/') if f.endswith('.zip')]

if zip_files:
    for zip_name in zip_files:
        src = f'/content/{zip_name}'
        dst = f'{backup_folder}/{zip_name}'
        size_mb = os.path.getsize(src) / (1024 * 1024)
        print(f"   Saving: {zip_name} ({size_mb:.1f} MB)")
        shutil.copy2(src, dst)
    print("=" * 60)
    print("✅ All ZIP files saved to Google Drive!")
    print(f"📂 Location: {backup_folder}")
else:
    print("❌ No ZIP files found!")


💾 Saving ZIP files to Google Drive...
   Saving: cow breed-3.zip (0.5 MB)
   Saving: cow breed-2.zip (0.2 MB)
   Saving: cow breed-1.zip (603.5 MB)
✅ All ZIP files saved to Google Drive!
📂 Location: /content/drive/MyDrive/cattle_breed_recognition/dataset_backup


In [8]:
# Alternative: Use unzip command for extraction
import os
import subprocess

print("📦 Extracting ZIP files using unzip command...")
print("=" * 60)

# Create temp directory
os.makedirs('/content/temp_images', exist_ok=True)

zip_files = ['/content/cow breed-1.zip', '/content/cow breed-2.zip', '/content/cow breed-3.zip']

for zip_path in zip_files:
    if os.path.exists(zip_path):
        size_mb = os.path.getsize(zip_path) / (1024 * 1024)
        print(f"\nExtracting: {os.path.basename(zip_path)} ({size_mb:.1f} MB)")

        try:
            # Use unzip command
            result = subprocess.run(
                ['unzip', '-o', '-q', zip_path, '-d', '/content/temp_images'],
                capture_output=True,
                text=True
            )

            if result.returncode == 0:
                print(f"   ✅ Extracted successfully")
            else:
                print(f"   ⚠️ Warning: {result.stderr[:100]}")
        except Exception as e:
            print(f"   ❌ Error: {e}")

# Count extracted images
print("\n" + "=" * 60)
print("📊 Counting extracted images...")

import glob
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.JPEG', '*.PNG']
all_images = []

for ext in image_extensions:
    all_images.extend(glob.glob(f'/content/temp_images/**/{ext}', recursive=True))

print(f"✅ Found {len(all_images)} images")


📦 Extracting ZIP files using unzip command...

Extracting: cow breed-1.zip (603.5 MB)
   ⚠️ Warning: error [/content/cow breed-1.zip]:  missing 24117248 bytes in zipfile
  (attempting to process anyway

Extracting: cow breed-2.zip (0.2 MB)
   ✅ Extracted successfully

Extracting: cow breed-3.zip (0.5 MB)
   ✅ Extracted successfully

📊 Counting extracted images...
✅ Found 34 images


In [9]:
import os

zip_path = '/content/cow breed-1.zip'
if os.path.exists(zip_path):
    size_mb = os.path.getsize(zip_path) / (1024 * 1024)
    print(f"📦 cow breed-1.zip: {size_mb:.1f} MB")

    if size_mb > 620:
        print("✅ File size looks correct!")
    else:
        print("⚠️ File may still be incomplete. Expected ~626 MB")
else:
    print("❌ File not found")


📦 cow breed-1.zip: 626.5 MB
✅ File size looks correct!


In [10]:
# Save the complete ZIP file to Google Drive
import os
import shutil

backup_folder = '/content/drive/MyDrive/cattle_breed_recognition/dataset_backup'
os.makedirs(backup_folder, exist_ok=True)

print("💾 Saving complete ZIP file to Google Drive...")

src = '/content/cow breed-1.zip'
dst = f'{backup_folder}/cow breed-1.zip'
size_mb = os.path.getsize(src) / (1024 * 1024)
print(f"   Saving: cow breed-1.zip ({size_mb:.1f} MB)")
shutil.copy2(src, dst)

print("✅ Saved to Google Drive!")


💾 Saving complete ZIP file to Google Drive...
   Saving: cow breed-1.zip (626.5 MB)
✅ Saved to Google Drive!


In [11]:
# Extract all ZIP files and create dataset
import os
import subprocess
import glob
import shutil
import random

DATASET_PATH = '/content/detection_dataset'

# Clean up previous extraction
if os.path.exists('/content/temp_images'):
    shutil.rmtree('/content/temp_images')
os.makedirs('/content/temp_images', exist_ok=True)

# Create dataset directories
for split in ['train', 'valid', 'test']:
    os.makedirs(f'{DATASET_PATH}/{split}/images', exist_ok=True)
    os.makedirs(f'{DATASET_PATH}/{split}/labels', exist_ok=True)

print("📦 Extracting all ZIP files...")
print("=" * 60)

zip_files = ['/content/cow breed-1.zip', '/content/cow breed-2.zip', '/content/cow breed-3.zip']

for zip_path in zip_files:
    if os.path.exists(zip_path):
        size_mb = os.path.getsize(zip_path) / (1024 * 1024)
        print(f"   Extracting: {os.path.basename(zip_path)} ({size_mb:.1f} MB)")

        result = subprocess.run(
            ['unzip', '-o', '-q', zip_path, '-d', '/content/temp_images'],
            capture_output=True, text=True
        )

        if result.returncode == 0:
            print(f"      ✅ Done")
        else:
            print(f"      ⚠️ {result.stderr[:50] if result.stderr else 'OK'}")

# Count images
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.JPEG', '*.PNG']
all_images = []
for ext in image_extensions:
    all_images.extend(glob.glob(f'/content/temp_images/**/{ext}', recursive=True))

print(f"\n✅ Found {len(all_images)} images")

# Split and copy
if len(all_images) > 0:
    random.shuffle(all_images)
    n = len(all_images)
    train_images = all_images[:int(n*0.7)]
    val_images = all_images[int(n*0.7):int(n*0.85)]
    test_images = all_images[int(n*0.85):]

    def copy_images(images, split):
        print(f"   Copying {len(images)} to {split}...")
        for i, img_path in enumerate(images):
            ext = os.path.splitext(img_path)[1]
            dest = f'{DATASET_PATH}/{split}/images/image_{i}{ext}'
            shutil.copy(img_path, dest)

            label_dest = f'{DATASET_PATH}/{split}/labels/image_{i}.txt'
            with open(label_dest, 'w') as f:
                f.write('0 0.5 0.5 0.8 0.8\n')

            if (i + 1) % 500 == 0:
                print(f"      Progress: {i + 1}/{len(images)}")

    copy_images(train_images, 'train')
    copy_images(val_images, 'valid')
    copy_images(test_images, 'test')

    shutil.rmtree('/content/temp_images', ignore_errors=True)

    print(f"\n✅ Dataset created:")
    print(f"   📂 Train: {len(train_images)} images")
    print(f"   📂 Valid: {len(val_images)} images")
    print(f"   📂 Test: {len(test_images)} images")


📦 Extracting all ZIP files...
   Extracting: cow breed-1.zip (626.5 MB)
      ✅ Done
   Extracting: cow breed-2.zip (0.2 MB)
      ✅ Done
   Extracting: cow breed-3.zip (0.5 MB)
      ✅ Done

✅ Found 2201 images
   Copying 1540 to train...
      Progress: 500/1540
      Progress: 1000/1540
      Progress: 1500/1540
   Copying 330 to valid...
   Copying 331 to test...

✅ Dataset created:
   📂 Train: 1540 images
   📂 Valid: 330 images
   📂 Test: 331 images


In [12]:
# COMPLETE: PyTorch Fix + Create data.yaml + Train Model
# Run this entire cell

# Step 1: Apply PyTorch Fix
import torch
import functools

_original_torch_load = torch.load

@functools.wraps(_original_torch_load)
def _patched_load(f, map_location=None, pickle_module=None, *, weights_only=None, mmap=None, **kwargs):
    return _original_torch_load(f, map_location=map_location, pickle_module=pickle_module,
                                 weights_only=False, mmap=mmap, **kwargs)

torch.load = _patched_load
print("✅ PyTorch patch applied!")

# Step 2: Create data.yaml
import os

yaml_content = """names:
- cattle
nc: 1
path: /content/detection_dataset
test: test/images
train: train/images
val: valid/images
"""

yaml_path = '/content/detection_dataset/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml created!")

# Step 3: Train model
print("\n🚀 Starting YOLOv8-Nano Training...")
print("=" * 60)
print(f"📊 Dataset: 1,540 train, 330 valid, 331 test")
print(f"🖥️ Device: CPU")
print(f"📐 Image size: 320x320")
print(f"🔄 Epochs: 20")
print("=" * 60)

from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data=yaml_path,
    epochs=20,
    imgsz=320,
    batch=16,
    name='cattle_detector',
    project='cattle_breed_recognition',
    device='cpu',
    patience=5,
    save=True,
    plots=True
)

print("\n✅ Training complete!")


✅ PyTorch patch applied!
✅ data.yaml created!

🚀 Starting YOLOv8-Nano Training...
📊 Dataset: 1,540 train, 330 valid, 331 test
🖥️ Device: CPU
📐 Image size: 320x320
🔄 Epochs: 20


100%|██████████| 6.23M/6.23M [00:00<00:00, 67.4MB/s]


RecursionError: maximum recursion depth exceeded

In [2]:
# Install onnxscript and export
!pip install onnxscript -q

print("✅ onnxscript installed!")

# Export to ONNX
from ultralytics import YOLO

model = YOLO('cattle_breed_recognition/cattle_detector/weights/best.pt')
model.export(format='onnx', imgsz=320, simplify=True)

print("✅ ONNX exported!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 15.0 MB/s eta 0:00:00
✅ onnxscript installed!


Ultralytics YOLOv8.0.196 🚀 Python-3.12.12 torch-2.10.0+cpu CPU (AMD EPYC 7B12)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'cattle_breed_recognition/cattle_detector/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 5, 2100) (5.9 MB)

ONNX: starting export with onnx 1.20.1 opset 10...
W0223 13:13:23.937000 25849 torch/onnx/_internal/exporter/_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 10 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0223 13:13:24.626000 25849 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' 

Applied 1 of general pattern rewrite rules.


ONNX: export success ✅ 4.8s, saved as 'cattle_breed_recognition/cattle_detector/weights/best.onnx' (11.5 MB)

Export complete (6.4s)
Results saved to /content/cattle_breed_recognition/cattle_detector/weights
Predict:         yolo predict task=detect model=cattle_breed_recognition/cattle_detector/weights/best.onnx imgsz=320  
Validate:        yolo val task=detect model=cattle_breed_recognition/cattle_detector/weights/best.onnx imgsz=320 data=/content/detection_dataset/data.yaml  
Visualize:       https://netron.app


✅ ONNX exported!


In [1]:
# COMPLETE TRAINING SCRIPT - Run after restart
# Do NOT run any previous cells

# Step 1: Install ultralytics
!pip install ultralytics -q
print("✅ Ultralytics installed!")

# Step 2: Apply PyTorch fix (ONLY ONCE)
import torch
import functools

# Get the ORIGINAL torch.load before any modifications
_torch_load = torch.load

# Create wrapper
@functools.wraps(_torch_load)
def _patched_torch_load(f, map_location=None, pickle_module=None, *, weights_only=None, mmap=None, **kwargs):
    return _torch_load(f, map_location=map_location, pickle_module=pickle_module,
                       weights_only=False, mmap=mmap, **kwargs)

# Apply patch ONCE
torch.load = _patched_torch_load
print("✅ PyTorch patch applied!")

# Step 3: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

# Step 4: Load ZIP files from Drive backup
import os
import shutil
import zipfile
import subprocess
import glob
import random

backup_folder = '/content/drive/MyDrive/cattle_breed_recognition/dataset_backup'
print("\n📂 Loading ZIP files from Google Drive...")

if os.path.exists(backup_folder):
    for f in os.listdir(backup_folder):
        if f.endswith('.zip'):
            src = os.path.join(backup_folder, f)
            dst = f'/content/{f}'
            if not os.path.exists(dst):
                size_mb = os.path.getsize(src) / (1024 * 1024)
                print(f"   Loading: {f} ({size_mb:.1f} MB)")
                shutil.copy2(src, dst)
            else:
                print(f"   Already exists: {f}")

# Step 5: Create dataset
DATASET_PATH = '/content/detection_dataset'

# Clean and create directories
if os.path.exists('/content/temp_images'):
    shutil.rmtree('/content/temp_images')
os.makedirs('/content/temp_images', exist_ok=True)

for split in ['train', 'valid', 'test']:
    os.makedirs(f'{DATASET_PATH}/{split}/images', exist_ok=True)
    os.makedirs(f'{DATASET_PATH}/{split}/labels', exist_ok=True)

print("\n📦 Extracting ZIP files...")
zip_files = ['/content/cow breed-1.zip', '/content/cow breed-2.zip', '/content/cow breed-3.zip']

for zip_path in zip_files:
    if os.path.exists(zip_path):
        print(f"   Extracting: {os.path.basename(zip_path)}")
        subprocess.run(['unzip', '-o', '-q', zip_path, '-d', '/content/temp_images'], capture_output=True)

# Count images
all_images = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.JPEG', '*.PNG']:
    all_images.extend(glob.glob(f'/content/temp_images/**/{ext}', recursive=True))

print(f"   Found {len(all_images)} images")

# Split and copy
if len(all_images) > 0:
    random.shuffle(all_images)
    n = len(all_images)

    for i, img_path in enumerate(all_images[:int(n*0.7)]):
        ext = os.path.splitext(img_path)[1]
        shutil.copy(img_path, f'{DATASET_PATH}/train/images/image_{i}{ext}')
        with open(f'{DATASET_PATH}/train/labels/image_{i}.txt', 'w') as f:
            f.write('0 0.5 0.5 0.8 0.8\n')

    for i, img_path in enumerate(all_images[int(n*0.7):int(n*0.85)]):
        ext = os.path.splitext(img_path)[1]
        shutil.copy(img_path, f'{DATASET_PATH}/valid/images/image_{i}{ext}')
        with open(f'{DATASET_PATH}/valid/labels/image_{i}.txt', 'w') as f:
            f.write('0 0.5 0.5 0.8 0.8\n')

    for i, img_path in enumerate(all_images[int(n*0.85):]):
        ext = os.path.splitext(img_path)[1]
        shutil.copy(img_path, f'{DATASET_PATH}/test/images/image_{i}{ext}')
        with open(f'{DATASET_PATH}/test/labels/image_{i}.txt', 'w') as f:
            f.write('0 0.5 0.5 0.8 0.8\n')

    shutil.rmtree('/content/temp_images', ignore_errors=True)
    print(f"✅ Dataset: {int(n*0.7)} train, {int(n*0.15)} valid, {n-int(n*0.85)} test")

# Step 6: Create data.yaml
yaml_content = f"""names:
- cattle
nc: 1
path: {DATASET_PATH}
test: test/images
train: train/images
val: valid/images
"""
with open(f'{DATASET_PATH}/data.yaml', 'w') as f:
    f.write(yaml_content)
print("✅ data.yaml created!")

# Step 7: Train
print("\n🚀 Training YOLOv8-Nano...")
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model.train(
    data=f'{DATASET_PATH}/data.yaml',
    epochs=20,
    imgsz=320,
    batch=16,
    name='cattle_detector',
    project='cattle_breed_recognition',
    device='cpu',
    patience=5,
    save=True
)

print("\n✅ Training complete!")

# Step 8: Export and Save
model.export(format='onnx', imgsz=320, simplify=True)
print("✅ ONNX exported!")

save_path = '/content/drive/MyDrive/cattle_breed_recognition/models'
os.makedirs(save_path, exist_ok=True)
!cp cattle_breed_recognition/cattle_detector/weights/best.pt {save_path}/cattle_detector.pt
!cp cattle_breed_recognition/cattle_detector/weights/best.onnx {save_path}/cattle_detector.onnx
print("✅ Model saved to Google Drive!")

print("\n🎉 ALL DONE!")


✅ Ultralytics installed!
✅ PyTorch patch applied!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted!

📂 Loading ZIP files from Google Drive...
   Already exists: cow breed-3.zip
   Already exists: cow breed-2.zip
   Already exists: cow breed-1.zip

📦 Extracting ZIP files...
   Extracting: cow breed-1.zip
   Extracting: cow breed-2.zip
   Extracting: cow breed-3.zip
   Found 2201 images
✅ Dataset: 1540 train, 330 valid, 331 test
✅ data.yaml created!

🚀 Training YOLOv8-Nano...


New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.12.12 torch-2.10.0+cpu CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/detection_dataset/data.yaml, epochs=20, patience=5, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=cattle_breed_recognition, name=cattle_detector, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visua

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py:238: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = amp.GradScaler(enabled=self.amp)
train: Scanning /content/detection_dataset/train/labels... 1875 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1875/1875 [00:02<00:00, 721.63it/s]
train: New cache created: /content/detection_dataset/train/labels.cache
/usr/local/lib/python3.12/dist-packages/ultralytics/data/augment.py:805: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0)]  # transforms
/usr/local/lib/python3.12/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray

lr/pg0,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▄██▁████████████████
metrics/mAP50-95(B),▂▃▄▁▆▅▇▆▆▇▇█████████
metrics/precision(B),▁▆▇▆▇▇▇██████▇██████
metrics/recall(B),▂▅██▁▅█▇▆███████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...



✅ Training complete!


Ultralytics YOLOv8.0.196 🚀 Python-3.12.12 torch-2.10.0+cpu CPU (AMD EPYC 7B12)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'cattle_breed_recognition/cattle_detector/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 5, 2100) (5.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxsim>=0.4.33', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 232.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 304.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 198.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 10.2s, installed 3 packages: ['onnx>=1.12.0', 'onnxsim>=0.4.33', 'onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.20.1 opset 10...
ONNX: export failure ❌ 10.6s: No module named 'onnxscript'


ModuleNotFoundError: No module named 'onnxscript'

In [3]:
# Save model to Google Drive
import os
import shutil

save_path = '/content/drive/MyDrive/cattle_breed_recognition/models'
os.makedirs(save_path, exist_ok=True)

# Copy files
shutil.copy('cattle_breed_recognition/cattle_detector/weights/best.pt', f'{save_path}/cattle_detector.pt')
shutil.copy('cattle_breed_recognition/cattle_detector/weights/best.onnx', f'{save_path}/cattle_detector.onnx')

print("✅ Model saved to Google Drive!")

# Verify
print("\n📁 Saved files:")
for f in os.listdir(save_path):
    fpath = os.path.join(save_path, f)
    if os.path.isfile(fpath):
        size_mb = os.path.getsize(fpath) / (1024 * 1024)
        print(f"   ✅ {f} ({size_mb:.1f} MB)")

print("\n🎉 ALL DONE! Model trained and saved!")


✅ Model saved to Google Drive!

📁 Saved files:
   ✅ cattle_detector.pt (5.9 MB)
   ✅ cattle_detector.onnx (11.5 MB)

🎉 ALL DONE! Model trained and saved!


## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

# Install YOLOv8
!pip install ultralytics==8.0.196 -q
!pip install roboflow -q

In [ ]:
import os
import yaml
import json
import shutil
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
from google.colab import drive

# Mount Google Drive for data storage
drive.mount('/content/drive')

# Set paths
BASE_PATH = '/content/drive/MyDrive/cattle_breed_recognition'
DATA_PATH = f'{BASE_PATH}/data'
MODELS_PATH = f'{BASE_PATH}/models'
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)

print(f"Base Path: {BASE_PATH}")
print(f"Data Path: {DATA_PATH}")
print(f"Models Path: {MODELS_PATH}")

## 2. Prepare Dataset

### Option A: Download from Roboflow (Recommended)

In [ ]:
# Option A: Download cattle detection dataset from Roboflow
# You can find cattle detection datasets at: https://universe.roboflow.com/

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("workspace-name").project("cattle-detection")
dataset = project.version(1).download("yolov8")

# The dataset will be downloaded in YOLOv8 format
DATASET_PATH = dataset.location
print(f"Dataset downloaded to: {DATASET_PATH}")

### Option B: Use Local Dataset

In [ ]:
# Option B: Upload your own dataset
# Dataset structure should be:
# dataset/
# ├── data.yaml
# ├── train/
# │   ├── images/
# │   └── labels/
# ├── valid/
# │   ├── images/
# │   └── labels/
# └── test/
#     ├── images/
#     └── labels/

# Upload dataset.zip to Colab and extract
# !unzip dataset.zip -d {DATA_PATH}

# DATASET_PATH = f'{DATA_PATH}/dataset'
# print(f"Dataset path: {DATASET_PATH}")

### Option C: Create Dataset from Kaggle Images

In [ ]:
# Option C: Create detection dataset from Kaggle breed images
# This creates bounding boxes around the entire image (simple approach)

def create_detection_dataset_from_classification(source_dir, output_dir, class_name='cattle'):
    """
    Create YOLO format detection dataset from classification images.
    Uses entire image as bounding box.

    Args:
        source_dir: Directory with class folders (classification format)
        output_dir: Output directory for YOLO format
        class_name: Class name for detection (cattle/buffalo)
    """
    import random

    # Create output directories
    for split in ['train', 'valid', 'test']:
        os.makedirs(f'{output_dir}/{split}/images', exist_ok=True)
        os.makedirs(f'{output_dir}/{split}/labels', exist_ok=True)

    # Collect all images
    all_images = []
    for breed_folder in os.listdir(source_dir):
        breed_path = os.path.join(source_dir, breed_folder)
        if os.path.isdir(breed_path):
            for img_name in os.listdir(breed_path):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    all_images.append({
                        'path': os.path.join(breed_path, img_name),
                        'name': f"{breed_folder}_{img_name}"
                    })

    print(f"Total images found: {len(all_images)}")

    # Shuffle and split
    random.shuffle(all_images)
    n = len(all_images)
    train_split = int(0.7 * n)
    val_split = int(0.85 * n)

    splits = {
        'train': all_images[:train_split],
        'valid': all_images[train_split:val_split],
        'test': all_images[val_split:]
    }

    # Process each split
    for split_name, images in splits.items():
        print(f"Processing {split_name}: {len(images)} images")

        for img_info in images:
            # Copy image
            img_path = img_info['path']
            img_name = img_info['name']
            dest_img = f'{output_dir}/{split_name}/images/{img_name}'
            shutil.copy(img_path, dest_img)

            # Get image dimensions
            img = cv2.imread(img_path)
            h, w = img.shape[:2]

            # Create YOLO format label (entire image as bbox)
            # YOLO format: class x_center y_center width height (normalized 0-1)
            # class 0 = cattle
            label_name = img_name.rsplit('.', 1)[0] + '.txt'
            label_path = f'{output_dir}/{split_name}/labels/{label_name}'

            # Entire image as bounding box
            x_center = 0.5
            y_center = 0.5
            width = 1.0
            height = 1.0

            with open(label_path, 'w') as f:
                f.write(f"0 {x_center} {y_center} {width} {height}\n")

    print(f"Dataset created at: {output_dir}")
    return output_dir

# Uncomment to use:
# SOURCE_DATA = f'{DATA_PATH}/raw/kaggle_cattle_images'
# OUTPUT_DATA = f'{DATA_PATH}/detection_dataset'
# create_detection_dataset_from_classification(SOURCE_DATA, OUTPUT_DATA)

## 3. Create data.yaml Configuration

In [ ]:
# Create data.yaml for YOLOv8

def create_data_yaml(dataset_path, output_path):
    """
    Create data.yaml configuration file for YOLOv8 training.
    """
    data_config = {
        'path': dataset_path,
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'nc': 1,  # Number of classes (1 = cattle/buffalo)
        'names': ['cattle']  # Class names
    }

    with open(output_path, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)

    print(f"Created data.yaml at: {output_path}")
    return output_path

# Create data.yaml
DATASET_PATH = f'{DATA_PATH}/detection_dataset'  # Update this path
YAML_PATH = create_data_yaml(DATASET_PATH, f'{DATASET_PATH}/data.yaml')

# Display config
with open(YAML_PATH, 'r') as f:
    print(f.read())

## 4. Train YOLOv8-Nano Model

In [ ]:
# Load YOLOv8-Nano model (pretrained on COCO)
model = YOLO('yolov8n.pt')  # Nano model

# Training parameters
training_params = {
    'data': YAML_PATH,
    'epochs': 100,              # Number of training epochs
    'imgsz': 416,               # Image size (smaller for mobile)
    'batch': 32,                # Batch size
    'name': 'cattle_detector',  # Experiment name
    'project': MODELS_PATH,     # Save directory
    'device': 0,                # GPU device
    'patience': 20,             # Early stopping patience
    'save': True,               # Save checkpoints
    'save_period': 10,          # Save every N epochs
    'workers': 4,               # Data loading workers
    'pretrained': True,         # Use pretrained weights
    'optimizer': 'auto',        # Optimizer (auto selects AdamW)
    'lr0': 0.01,                # Initial learning rate
    'lrf': 0.01,                # Final learning rate
    'momentum': 0.937,          # SGD momentum
    'weight_decay': 0.0005,     # Weight decay
    'warmup_epochs': 3,         # Warmup epochs
    'warmup_momentum': 0.8,     # Warmup momentum
    'warmup_bias_lr': 0.1,      # Warmup bias learning rate
    'box': 7.5,                 # Box loss gain
    'cls': 0.5,                 # Classification loss gain
    'dfl': 1.5,                 # Distribution focal loss gain
    'pose': 12.0,               # Pose loss gain
    'kobj': 1.0,                # Keypoint objectness loss gain
    'label_smoothing': 0.0,     # Label smoothing
    'nbs': 64,                  # Nominal batch size
    'hsv_h': 0.015,             # HSV-Hue augmentation
    'hsv_s': 0.7,               # HSV-Saturation augmentation
    'hsv_v': 0.4,               # HSV-Value augmentation
    'degrees': 15.0,            # Rotation augmentation (+/- deg)
    'translate': 0.1,           # Translation augmentation (+/- fraction)
    'scale': 0.5,               # Scaling augmentation (+/- gain)
    'shear': 0.0,               # Shear augmentation (+/- deg)
    'perspective': 0.0,         # Perspective augmentation (+/- fraction)
    'flipud': 0.0,              # Flip up-down probability
    'fliplr': 0.5,              # Flip left-right probability
    'mosaic': 1.0,              # Mosaic augmentation probability
    'mixup': 0.0,               # Mixup augmentation probability
    'copy_paste': 0.0,          # Copy-paste augmentation probability
    'auto_augment': 'randaugment',  # Auto augmentation policy
    'erasing': 0.4,             # Random erasing probability
    'crop_fraction': 1.0,       # Image crop fraction
}

print("Starting YOLOv8-Nano training...")
print(f"Dataset: {YAML_PATH}")
print(f"Image size: 416x416")
print(f"Epochs: 100")

In [ ]:
# Start training
results = model.train(**training_params)

## 5. Evaluate Model Performance

In [ ]:
# Validate the model
metrics = model.val()

print("\n=== Validation Metrics ===")
print(f"mAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

In [ ]:
# Plot training results
from IPython.display import Image, display

# Display results plot
results_path = f'{MODELS_PATH}/cattle_detector'
if os.path.exists(f'{results_path}/results.png'):
    display(Image(filename=f'{results_path}/results.png', width=800))

In [ ]:
# Test on sample images
test_images_path = f'{DATASET_PATH}/test/images'
test_images = os.listdir(test_images_path)[:5]

for img_name in test_images:
    img_path = os.path.join(test_images_path, img_name)
    results = model.predict(img_path, save=True, conf=0.25)

    # Display result
    result_img = f'{results[0].save_dir}/{img_name}'
    if os.path.exists(result_img):
        display(Image(filename=result_img, width=400))

## 6. Export to TFLite with INT8 Quantization

In [ ]:
# Export to TFLite format
# YOLOv8 supports direct TFLite export

# Load best model
best_model = YOLO(f'{MODELS_PATH}/cattle_detector/weights/best.pt')

# Export to TFLite
best_model.export(
    format='tflite',
    imgsz=416,
    int8=True,           # INT8 quantization
    data=YAML_PATH,      # Dataset for calibration
    batch=1,
    optimize=True,       # Optimize for mobile
    simplify=True,       # Simplify model
    opset=12,            # ONNX opset version
    workspace=4,         # TensorRT workspace size (GB)
)

print("Model exported to TFLite with INT8 quantization!")

In [ ]:
# Check exported model size
import glob

tflite_files = glob.glob(f'{MODELS_PATH}/cattle_detector/weights/*.tflite')
for tflite_file in tflite_files:
    size_mb = os.path.getsize(tflite_file) / (1024 * 1024)
    print(f"{os.path.basename(tflite_file)}: {size_mb:.2f} MB")

## 7. Test TFLite Model

In [ ]:
# Test TFLite model inference
import numpy as np
import tensorflow as tf

def test_tflite_model(tflite_path, test_image_path):
    """
    Test TFLite model inference on a single image.
    """
    # Load TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    # Get input/output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print(f"Input shape: {input_details[0]['shape']}")
    print(f"Input dtype: {input_details[0]['dtype']}")
    print(f"Output shape: {output_details[0]['shape']}")

    # Load and preprocess image
    img = cv2.imread(test_image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (416, 416))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)

    # Run inference
    import time
    start_time = time.time()

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    inference_time = (time.time() - start_time) * 1000
    print(f"\nInference time: {inference_time:.2f} ms")
    print(f"Output shape: {output.shape}")

    return output

# Test on sample image
tflite_path = glob.glob(f'{MODELS_PATH}/cattle_detector/weights/*_int8.tflite')[0]
test_image = os.path.join(test_images_path, test_images[0])

print(f"Testing TFLite model: {tflite_path}")
print(f"Test image: {test_image}")

output = test_tflite_model(tflite_path, test_image)

## 8. Save Final Model

In [ ]:
# Copy final models to output directory
OUTPUT_DIR = f'{MODELS_PATH}/final'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Copy best PyTorch model
shutil.copy(
    f'{MODELS_PATH}/cattle_detector/weights/best.pt',
    f'{OUTPUT_DIR}/yolov8_nano_cattle_detector.pt'
)

# Copy TFLite model
for tflite_file in tflite_files:
    shutil.copy(tflite_file, OUTPUT_DIR)

print(f"Final models saved to: {OUTPUT_DIR}")
print("\nFiles:")
for f in os.listdir(OUTPUT_DIR):
    size_mb = os.path.getsize(os.path.join(OUTPUT_DIR, f)) / (1024 * 1024)
    print(f"  {f}: {size_mb:.2f} MB")

## 9. Model Summary

In [ ]:
print("="*60)
print("YOLOv8-Nano Detection Model Training Complete!")
print("="*60)
print(f"\nModel: YOLOv8-Nano")
print(f"Task: Cattle/Buffalo Detection")
print(f"Input Size: 416x416")
print(f"\nPerformance Metrics:")
print(f"  mAP@50: {metrics.box.map50:.4f}")
print(f"  mAP@50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")
print(f"\nModel Files:")
print(f"  PyTorch: yolov8_nano_cattle_detector.pt")
print(f"  TFLite INT8: *_int8.tflite (~5 MB)")
print(f"\nReady for Stage 2: Breed Classification with EfficientNet-B0")
print("="*60)